In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!rm -rf /content/gdrive
!ln -s "/gdrive/My Drive" /content/gdrive
%cd /content/gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive


In [ ]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [ ]:
twitter=pd.read_csv("Cyberbullying_twitter_## only labeling_racism_200805.csv", encoding='latin1')

In [ ]:
twitter.head(10)

,sentence,sentiment
0,RT @colonelkickhead: Another bloody instant re...,none
1,@azzamalirhabi @JihadiA8 This video of the Pes...,none
2,Oh really? No more instant restaurants? THAT'S...,none
3,RT @Benfrancisallen: It hasn't been a good few...,none
4,RT @NoToFeminism: I donât need femisnsn beca...,none
5,@MariachiMacabre 19% is not the vast majority,none
6,@DianH4 @ExposeFalsehood And it is Muslims who...,racism
7,@truaemusic @mattybboi83 @Number10gov Capital ...,racism
8,Gather round kids. It's story time brought to ...,none
9,Did you hear that? That's the sound of a bunch...,none


In [ ]:
data = twitter[['sentence','sentiment']]

In [ ]:
import string

data['sentence'] = data['sentence'].apply(lambda x: x.lower())
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"\brt\b", "", x).strip()))
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"(https?|http)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]", "", x)))
data['sentence'] = data['sentence'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))
print(data[ data['sentiment'] == 'none'].size)
print(data[ data['sentiment'] == 'racism'].size)

21504
3780


In [ ]:
def text_to_wordlist(text, remove_stopwords=True):    

    text = text.lower().split()
    
    if remove_stopwords:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    return(text)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data['sentence'] = data.apply(lambda row: text_to_wordlist(row.sentence).split(), axis=1)
data.head(10)

,sentence,sentiment
0,"[colonelkickhead, another, bloody, instant, re...",none
1,"[azzamalirhabi, jihadia8, this, video, of, the...",none
2,"[oh, really, no, more, instant, restaurant, th...",none
3,"[benfrancisallen, it, hasnt, been, a, good, fe...",none
4,"[notofeminism, i, donât, need, femisnsn, bec...",none
5,"[mariachimacabre, 19, is, not, the, vast, majo...",none
6,"[dianh4, exposefalsehood, and, it, is, muslim,...",racism
7,"[truaemusic, mattybboi83, number10gov, capital...",racism
8,"[gather, round, kid, it, story, time, brought,...",none
9,"[did, you, hear, that, thats, the, sound, of, ...",none


In [ ]:
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(data['sentence'].values)
X = tokenizer.texts_to_sequences(data['sentence'].values)
X = pad_sequences(X, maxlen=26)

In [ ]:
Y = pd.get_dummies(data['sentiment']).values

In [ ]:
print(X.shape)

(12642, 26)


In [ ]:
word_index = tokenizer.word_index
print(word_index)

{'the': 1, 'to': 2, 'a': 3, 'mkr': 4, 'and': 5, 'of': 6, 'is': 7, 'i': 8, 'you': 9, 'that': 10, 'it': 11, 'in': 12, 'are': 13, 'for': 14, 'not': 15, 'this': 16, 'they': 17, 'on': 18, 'have': 19, 'be': 20, 'with': 21, 'but': 22, 'islam': 23, 'so': 24, 'about': 25, 'muslim': 26, 'all': 27, 'wa': 28, 'my': 29, 'no': 30, 'like': 31, 'what': 32, 'just': 33, 'your': 34, 'im': 35, 'kat': 36, 'if': 37, 'dont': 38, 'me': 39, 'do': 40, 'at': 41, 'people': 42, 'there': 43, 'by': 44, 'from': 45, 'woman': 46, 'one': 47, 'we': 48, 'amp': 49, 'get': 50, 'how': 51, 'can': 52, 'their': 53, 'them': 54, 'up': 55, 'isi': 56, 'he': 57, 'will': 58, 'more': 59, 'an': 60, 'who': 61, 'andre': 62, 'ha': 63, 'or': 64, 'would': 65, 'out': 66, 'think': 67, 'time': 68, 'when': 69, 'because': 70, 'now': 71, 'go': 72, 'were': 73, 'know': 74, 'make': 75, 'why': 76, 'want': 77, 'going': 78, 'good': 79, 'thing': 80, 'only': 81, 'really': 82, 'say': 83, 'â\x80¦': 84, 'been': 85, 'see': 86, 'cant': 87, 'should': 88, 'bein

In [ ]:
from sklearn.model_selection import KFold

skf = KFold(n_splits=5, shuffle=True)
accuracy = []
precision = []
recall = []
f1_score = []

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/gdrive/glove.twitter.27B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
from keras.layers import Bidirectional, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from sklearn import metrics
from sklearn.utils import shuffle

X, Y = shuffle(X, Y, random_state=42)

embed_dim = 50
lstm_out = 50
for train, validation in skf.split(X, Y):
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index) + 1, embed_dim, weights=[embedding_matrix], input_length = X.shape[1]))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Bidirectional(LSTM(lstm_out, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model.fit(X[train], Y[train], epochs=10, batch_size=128)
    k_accuracy = '%.4f' % (model.evaluate(X[validation], Y[validation])[1])
    predict_test = (model.predict(X[validation]) >= 0.5) 
    k_precision = metrics.precision_score(Y[validation], predict_test, average=None)
    k_recall = metrics.recall_score(Y[validation], predict_test, average=None)
    k_f1_score = metrics.f1_score(Y[validation], predict_test, average=None)
    accuracy.append(k_accuracy)
    precision.append(k_precision)
    recall.append(k_recall)
    f1_score.append(k_f1_score)

Epoch 1/10
80/80 [==============================] - 6s 15ms/step - loss: 0.4528 - accuracy: 0.8227
Epoch 2/10
80/80 [==============================] - 1s 15ms/step - loss: 0.2154 - accuracy: 0.9041
Epoch 3/10
80/80 [==============================] - 1s 14ms/step - loss: 0.1702 - accuracy: 0.9297
Epoch 4/10
80/80 [==============================] - 1s 14ms/step - loss: 0.1558 - accuracy: 0.9319
Epoch 5/10
80/80 [==============================] - 1s 13ms/step - loss: 0.1346 - accuracy: 0.9405
Epoch 6/10
80/80 [==============================] - 1s 13ms/step - loss: 0.1112 - accuracy: 0.9518
Epoch 7/10
80/80 [==============================] - 1s 13ms/step - loss: 0.1050 - accuracy: 0.9557
Epoch 8/10
80/80 [==============================] - 1s 13ms/step - loss: 0.0849 - accuracy: 0.9684
Epoch 9/10
80/80 [==============================] - 1s 14ms/step - loss: 0.0771 - accuracy: 0.9690
Epoch 10/10
80/80 [==============================] - 1s 3ms/step - loss: 0.2408 - accuracy: 0.9272
Epoch 1/10

In [ ]:
def average(list):
    return (sum(list) / len(list))

In [ ]:
accuracy = list(map(float, accuracy))

In [ ]:
print(average(accuracy))

0.92698


In [ ]:
print(average(precision))

[0.96358283 0.73932995]


In [ ]:
print(average(recall))

[0.95007382 0.79349643]


In [ ]:
print(average(f1_score))

[0.95672777 0.76435712]
